In [2]:
X_train = np.load('../ECGdata/normal.npy')
temp = np.load('../ECGdata/abnormal.npy')
X_train = np.vstack((X_train,temp))

Y_train = np.load('../ECGdata/label_1.npy')
temp = np.load('../ECGdata/label_0.npy')
Y_train = np.vstack((Y_train,temp))

print X_train.shape
print Y_train.shape

(359796, 1700)
(29983, 2)


In [3]:
X_train = X_train.reshape(-1,12*1700)
print X_train.shape

(29983, 20400)


In [5]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.2, random_state=2018)
print x_train.shape

(23986, 20400)


In [6]:
class Attention_layer(Layer):
    """
        Attention operation, with a context/query vector, for temporal data.
        Supports Masking.
        Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
        "Hierarchical Attention Networks for Document Classification"
        by using a context vector to assist the attention
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(AttentionWithContext())
        """
 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
 
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
 
        if self.bias:
            uit += self.b
 
        uit = K.tanh(uit)
 
        a = K.exp(uit)
 
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
 
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        # a = K.expand_dims(a)
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (-1, input_shape[-1])

In [7]:
def conv_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    x = Activation('relu')(input)
    x = Convolution1D(nb_filter, (9), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    return x

def dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    feature_list = [x]

    for i in range(nb_layers):
        x = conv_block(x, growth_rate, dropout_rate, weight_decay)
        feature_list.append(x)
        x = Concatenate(axis=concat_axis)(feature_list)
        nb_filter += growth_rate

    return x, nb_filter

def transition_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = Convolution1D(nb_filter, (1), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(input)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling1D((2), strides=(1))(x)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)

    return x

def createDenseNet(nb_classes, img_dim, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=16, dropout_rate=None,
                     weight_decay=1E-4, verbose=True):

    model_input = Input(shape=img_dim)

    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"

    # layers in each dense block
    nb_layers = int((depth - 4) / 3)

    # Initial convolution
    x = Convolution1D(nb_filter, (9), kernel_initializer="he_uniform", padding="same", name="initial_conv2D", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(model_input)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                            beta_regularizer=l2(weight_decay))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)
        # add transition_block
        x = transition_block(x, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)

    # The last dense_block does not have a transition_block
    x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = Activation('relu')(x)
#     print x.shape
#     x = AveragePooling2D((1, 25), strides=(1, 25))(x)
    x = GlobalAveragePooling1D()(x)
#     print x.shape
#     x = Attention_layer()(x)
    x = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(x)

    densenet = Model(inputs=model_input, outputs=x)

    if verbose: 
        print("DenseNet-%d-%d created." % (depth, growth_rate))

    return densenet



#define DenseNet parms
ROWS = 12
COLS = 1700
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS)
densenet_depth = 40
densenet_growth_rate = 12

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim,depth=densenet_depth,
                  growth_rate = densenet_growth_rate)

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
checkpoint = keras.callbacks.ModelCheckpoint('DenseNet.h5', monitor='val_acc', verbose=1, save_best_only=True)

xtrain = x_train.reshape(-1,12,1700)
xvalid = x_valid.reshape(-1,12,1700)
model.fit(xtrain, y_train, batch_size=32, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [checkpoint])

DenseNet-40-12 created.
Train on 23986 samples, validate on 5997 samples
Epoch 1/50
23986/23986 [==============================] - 40s 2ms/step - loss: 0.8947 - acc: 0.5953 - val_loss: 0.8622 - val_acc: 0.6560

Epoch 00001: val_acc improved from -inf to 0.65599, saving model to DenseNet.h5
Epoch 2/50
23986/23986 [==============================] - 35s 1ms/step - loss: 0.8224 - acc: 0.6881 - val_loss: 0.8463 - val_acc: 0.6695

Epoch 00002: val_acc improved from 0.65599 to 0.66950, saving model to DenseNet.h5
Epoch 3/50
23986/23986 [==============================] - 35s 1ms/step - loss: 0.7762 - acc: 0.7262 - val_loss: 0.8230 - val_acc: 0.6887

Epoch 00003: val_acc improved from 0.66950 to 0.68868, saving model to DenseNet.h5
Epoch 4/50
23986/23986 [==============================] - 35s 1ms/step - loss: 0.7365 - acc: 0.7545 - val_loss: 0.8602 - val_acc: 0.6870

Epoch 00004: val_acc did not improve from 0.68868
Epoch 5/50
23986/23986 [==============================] - 34s 1ms/step - loss: 

KeyboardInterrupt: 

In [11]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
def conv_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    x = Activation('relu')(input)
    x = Convolution2D(nb_filter, (1, 9), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    return x

def dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    feature_list = [x]

    for i in range(nb_layers):
        x = conv_block(x, growth_rate, dropout_rate, weight_decay)
        feature_list.append(x)
        x = Concatenate(axis=concat_axis)(feature_list)
        nb_filter += growth_rate

    return x, nb_filter

def transition_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = Convolution2D(nb_filter, (1, 1), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(input)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling2D((1, 2), strides=(1, 2))(x)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)

    return x

def createDenseNet(nb_classes, img_dim, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=16, dropout_rate=None,
                     weight_decay=1E-4, verbose=True):

    model_input = Input(shape=img_dim)

    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"

    # layers in each dense block
    nb_layers = int((depth - 4) / 3)

    # Initial convolution
    x = Convolution2D(nb_filter, (1, 9), kernel_initializer="he_uniform", padding="same", name="initial_conv2D", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(model_input)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                            beta_regularizer=l2(weight_decay))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)
        # add transition_block
        x = transition_block(x, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)

    # The last dense_block does not have a transition_block
    x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(x)

    densenet = Model(inputs=model_input, outputs=x)

    if verbose: 
        print("DenseNet-%d-%d created." % (depth, growth_rate))

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 1700
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)
densenet_depth = 40
densenet_growth_rate = 12

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim,depth=densenet_depth,
                  growth_rate = densenet_growth_rate)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
xtrain = x_train.reshape(-1,12,1700,1)
xvalid = x_valid.reshape(-1,12,1700,1)
model.fit(xtrain, y_train, batch_size=10, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

DenseNet-40-12 created.
Train on 23986 samples, validate on 5997 samples
Epoch 1/50
23986/23986 [==============================] - 691s 29ms/step - loss: 0.8168 - acc: 0.6798 - val_loss: 0.8734 - val_acc: 0.6223
Epoch 2/50
23986/23986 [==============================] - 687s 29ms/step - loss: 0.7484 - acc: 0.7192 - val_loss: 1.0302 - val_acc: 0.5793
Epoch 3/50
23986/23986 [==============================] - 686s 29ms/step - loss: 0.7131 - acc: 0.7332 - val_loss: 0.7198 - val_acc: 0.7310
Epoch 4/50
23986/23986 [==============================] - 683s 28ms/step - loss: 0.6843 - acc: 0.7398 - val_loss: 0.6623 - val_acc: 0.7620
Epoch 5/50
23986/23986 [==============================] - 685s 29ms/step - loss: 0.6616 - acc: 0.7468 - val_loss: 0.6137 - val_acc: 0.7826
Epoch 6/50
23986/23986 [==============================] - 685s 29ms/step - loss: 0.6355 - acc: 0.7578 - val_loss: 0.6015 - val_acc: 0.7784
Epoch 7/50
23986/23986 [==============================] - 684s 29ms/step - loss: 0.6135 - acc

In [13]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
def conv_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    x = Activation('relu')(input)
    x = Convolution2D(nb_filter, (1, 9), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    return x

def dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    feature_list = [x]

    for i in range(nb_layers):
        x = conv_block(x, growth_rate, dropout_rate, weight_decay)
        feature_list.append(x)
        x = Concatenate(axis=concat_axis)(feature_list)
        nb_filter += growth_rate

    return x, nb_filter

def transition_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = Convolution2D(nb_filter, (1, 1), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(input)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling2D((1, 2), strides=(1, 2))(x)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)

    return x

def createDenseNet(nb_classes, img_dim, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=16, dropout_rate=None,
                     weight_decay=1E-4, verbose=True):

    model_input = Input(shape=img_dim)

    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"

    # layers in each dense block
    nb_layers = int((depth - 4) / 3)

    # Initial convolution
    x = Convolution2D(nb_filter, (1, 9), kernel_initializer="he_uniform", padding="same", name="initial_conv2D", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(model_input)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                            beta_regularizer=l2(weight_decay))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)
        # add transition_block
        x = transition_block(x, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)

    # The last dense_block does not have a transition_block
    x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(x)

    densenet = Model(inputs=model_input, outputs=x)

    if verbose: 
        print("DenseNet-%d-%d created." % (depth, growth_rate))

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 1700
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)
densenet_depth = 40
densenet_growth_rate = 12

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim,depth=densenet_depth,
                  growth_rate = densenet_growth_rate)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
xtrain = x_train.reshape(-1,12,1700,1)
xvalid = x_valid.reshape(-1,12,1700,1)
model.fit(xtrain, y_train, batch_size=24, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

DenseNet-40-12 created.
Train on 23986 samples, validate on 5997 samples
Epoch 1/50
23986/23986 [==============================] - 661s 28ms/step - loss: 0.8067 - acc: 0.6932 - val_loss: 1.0090 - val_acc: 0.5813
Epoch 2/50
23986/23986 [==============================] - 657s 27ms/step - loss: 0.7553 - acc: 0.7229 - val_loss: 0.7649 - val_acc: 0.7144
Epoch 3/50
23986/23986 [==============================] - 655s 27ms/step - loss: 0.7303 - acc: 0.7378 - val_loss: 0.8906 - val_acc: 0.6026
Epoch 4/50
23986/23986 [==============================] - 656s 27ms/step - loss: 0.7083 - acc: 0.7494 - val_loss: 0.8943 - val_acc: 0.6930
Epoch 5/50
23986/23986 [==============================] - 656s 27ms/step - loss: 0.6875 - acc: 0.7566 - val_loss: 0.6657 - val_acc: 0.7696
Epoch 6/50
23986/23986 [==============================] - 655s 27ms/step - loss: 0.6686 - acc: 0.7634 - val_loss: 0.6662 - val_acc: 0.7642
Epoch 7/50
23986/23986 [==============================] - 656s 27ms/step - loss: 0.6572 - acc

In [20]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
batch_size = 32
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2), input_shape=(12* 1700,1)))
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(7)))
model.add(Dropout(0.25))

model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(LSTM(32, return_sequences=True,
               batch_input_shape=(batch_size, 12, 1700)))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(32, return_sequences=True))
model.add(Attention_layer())

model.add(Dense(2, kernel_initializer='glorot_uniform', activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

xtrain = x_train.reshape(-1,12*1700,1)
xvalid = x_valid.reshape(-1,12*1700,1)

model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])
# score = model.evaluate(x_test, y_test, batch_size=batch_size)
# print model.predict(x_test)
# print K.eval(categorical_accuracy(y_test, model.predict(x_test)))
# print score

Tensor("attention_layer_7/div:0", shape=(?, 77, 32), dtype=float32)
Tensor("lstm_7/transpose_1:0", shape=(?, ?, 32), dtype=float32)
Tensor("attention_layer_7/mul:0", shape=(?, 77, 32), dtype=float32)
Train on 23986 samples, validate on 5997 samples
Epoch 1/50
23986/23986 [==============================] - 138s 6ms/step - loss: 0.6928 - acc: 0.5098 - val_loss: 0.6868 - val_acc: 0.5573
Epoch 2/50
23986/23986 [==============================] - 136s 6ms/step - loss: 0.6174 - acc: 0.6488 - val_loss: 0.5560 - val_acc: 0.7174
Epoch 3/50
23986/23986 [==============================] - 136s 6ms/step - loss: 0.5044 - acc: 0.7541 - val_loss: 0.4734 - val_acc: 0.7782
Epoch 4/50
23986/23986 [==============================] - 135s 6ms/step - loss: 0.4766 - acc: 0.7699 - val_loss: 0.5252 - val_acc: 0.7560
Epoch 5/50
23986/23986 [==============================] - 136s 6ms/step - loss: 0.4614 - acc: 0.7815 - val_loss: 0.4823 - val_acc: 0.7727
Epoch 6/50
23986/23986 [==============================] - 137

In [25]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
batch_size = 32
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2), input_shape=(12* 1700,1)))
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(7)))
model.add(Dropout(0.25))

model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(LSTM(32, return_sequences=True,
               batch_input_shape=(batch_size, 12, 1700)))
model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(32, return_sequences=True))
model.add(Attention_layer())

model.add(Dense(2, kernel_initializer='glorot_uniform', activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

xtrain = x_train.reshape(-1,12*1700,1)
xvalid = x_valid.reshape(-1,12*1700,1)

model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])
# score = model.evaluate(x_test, y_test, batch_size=batch_size)
# print model.predict(x_test)
# print K.eval(categorical_accuracy(y_test, model.predict(x_test)))
# print score

Tensor("attention_layer_9/div:0", shape=(?, 77, 32), dtype=float32)
Tensor("lstm_11/transpose_1:0", shape=(?, ?, 32), dtype=float32)
Tensor("attention_layer_9/mul:0", shape=(?, 77, 32), dtype=float32)
Train on 23986 samples, validate on 5997 samples
Epoch 1/50
23986/23986 [==============================] - 252s 10ms/step - loss: 0.6929 - acc: 0.5135 - val_loss: 0.6885 - val_acc: 0.5861
Epoch 2/50
23986/23986 [==============================] - 246s 10ms/step - loss: 0.6051 - acc: 0.6643 - val_loss: 0.5678 - val_acc: 0.6923
Epoch 3/50
23986/23986 [==============================] - 247s 10ms/step - loss: 0.5068 - acc: 0.7568 - val_loss: 0.5035 - val_acc: 0.7637
Epoch 4/50
23986/23986 [==============================] - 247s 10ms/step - loss: 0.4836 - acc: 0.7710 - val_loss: 0.4786 - val_acc: 0.7734
Epoch 5/50
23986/23986 [==============================] - 245s 10ms/step - loss: 0.4695 - acc: 0.7776 - val_loss: 0.4639 - val_acc: 0.7869
Epoch 6/50
23986/23986 [==============================]

In [26]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
batch_size = 32
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2), input_shape=(12* 1700,1)))
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(7)))
model.add(Dropout(0.25))

model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(LSTM(32, return_sequences=True,
               batch_input_shape=(batch_size, 12, 1700)))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(32, return_sequences=True))
model.add(Attention_layer())

model.add(Dense(2, kernel_initializer='glorot_uniform', activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

xtrain = x_train.reshape(-1,12*1700,1)
xvalid = x_valid.reshape(-1,12*1700,1)

model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])
# score = model.evaluate(x_test, y_test, batch_size=batch_size)
# print model.predict(x_test)
# print K.eval(categorical_accuracy(y_test, model.predict(x_test)))
# print score

Tensor("attention_layer_10/div:0", shape=(?, 77, 32), dtype=float32)
Tensor("lstm_14/transpose_1:0", shape=(?, ?, 32), dtype=float32)
Tensor("attention_layer_10/mul:0", shape=(?, 77, 32), dtype=float32)
Train on 23986 samples, validate on 5997 samples
Epoch 1/50
23986/23986 [==============================] - 361s 15ms/step - loss: 0.6939 - acc: 0.4967 - val_loss: 0.6932 - val_acc: 0.4989
Epoch 2/50
23986/23986 [==============================] - 356s 15ms/step - loss: 0.6937 - acc: 0.5023 - val_loss: 0.6932 - val_acc: 0.5011
Epoch 3/50
23986/23986 [==============================] - 362s 15ms/step - loss: 0.6937 - acc: 0.4978 - val_loss: 0.6935 - val_acc: 0.4989
Epoch 4/50
23986/23986 [==============================] - 364s 15ms/step - loss: 0.6938 - acc: 0.4974 - val_loss: 0.6948 - val_acc: 0.5011
Epoch 5/50
23986/23986 [==============================] - 354s 15ms/step - loss: 0.6935 - acc: 0.5039 - val_loss: 0.6934 - val_acc: 0.4991
Epoch 6/50
23986/23986 [=============================

In [27]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
batch_size = 32
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2), input_shape=(12* 1700,1)))
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(7)))
model.add(Dropout(0.25))

model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(LSTM(32, return_sequences=True,
               batch_input_shape=(batch_size, 12, 1700)))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(Attention_layer())

model.add(Dense(2, kernel_initializer='glorot_uniform', activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

xtrain = x_train.reshape(-1,12*1700,1)
xvalid = x_valid.reshape(-1,12*1700,1)

model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])
# score = model.evaluate(x_test, y_test, batch_size=batch_size)
# print model.predict(x_test)
# print K.eval(categorical_accuracy(y_test, model.predict(x_test)))
# print score

Tensor("attention_layer_11/div:0", shape=(?, 77, 32), dtype=float32)
Tensor("lstm_18/transpose_1:0", shape=(?, ?, 32), dtype=float32)
Tensor("attention_layer_11/mul:0", shape=(?, 77, 32), dtype=float32)
Train on 23986 samples, validate on 5997 samples
Epoch 1/50
23986/23986 [==============================] - 476s 20ms/step - loss: 0.6941 - acc: 0.4958 - val_loss: 0.6950 - val_acc: 0.4989
Epoch 2/50
23986/23986 [==============================] - 467s 19ms/step - loss: 0.6940 - acc: 0.4985 - val_loss: 0.6931 - val_acc: 0.5021
Epoch 3/50
23986/23986 [==============================] - 465s 19ms/step - loss: 0.6938 - acc: 0.5001 - val_loss: 0.6938 - val_acc: 0.4989
Epoch 4/50
23986/23986 [==============================] - 467s 19ms/step - loss: 0.6937 - acc: 0.5013 - val_loss: 0.6932 - val_acc: 0.5011
Epoch 5/50
23986/23986 [==============================] - 461s 19ms/step - loss: 0.6939 - acc: 0.4957 - val_loss: 0.6936 - val_acc: 0.4989
Epoch 6/50
23986/23986 [=============================